In [1]:
import wind_calculations as wc
import panel as pn


In [2]:
pn.extension()


def info_icon(text):
    return pn.pane.Markdown(
        f"<span title='{text}' style='cursor:help; color:#1976d2; font-size:18px;'>ℹ️</span>",
        width=20
    )


def calculate_outputs(rotor_diameter, available_area_km2, spacing_factor, efficiency):
    air_density = wc.air_density_lookup[rotor_diameter]
    wind_speed = wc.wind_speed_lookup[rotor_diameter]
    pd = wc.annual_power_density(wind_speed, air_density)
    area = wc.swept_area(rotor_diameter)
    pk = wc.power_kw(pd, rotor_diameter)
    energy_non_derated = wc.annual_energy_output(pk)
    energy_derated = wc.derated_annual_energy_output(pk, efficiency)
    turbines = wc.possible_turbine_installations(available_area_km2, rotor_diameter, spacing_factor)
    # Derived spacing (center-to-center) between turbines in meters
    spacing_m = rotor_diameter * spacing_factor
    # Site totals
    site_power = pk * turbines
    site_energy_non_derated = energy_non_derated * turbines
    site_energy_derated = energy_derated * turbines
    return {
        'air_density': air_density,
        'wind_speed': wind_speed,
        'power_density': pd,
        'swept_area': area,
        'power_kw': pk,
        'energy_non_derated': energy_non_derated,
        'energy_derated': energy_derated,
        'turbines': turbines,
        'turbine_spacing_m': spacing_m,
        'site_power': site_power,
        'site_energy_non_derated': site_energy_non_derated,
        'site_energy_derated': site_energy_derated,
        'efficiency': efficiency
    }


# Panel widgets
rotor_diameter_dropdown = pn.widgets.Select(
    name="Turbine Rotor Diameter (m)",
    options=[100, 150, 200, 250],
    value=100
)
available_area_input = pn.widgets.IntSlider(
    name="Available Area (km²)",
    start=100,
    end=1000,
    step=50,
    value=200
)

# Spacing factor slider
spacing_factor_slider = pn.widgets.FloatSlider(
    name="Turbine Density Factor (Spacing Factor)",
    start=3.0,
    end=10.0,
    step=0.25,
    value=6.0,
    format='0.00'
)

# Efficiency (derating) slider
efficiency_slider = pn.widgets.FloatSlider(
    name="Efficiency (derating)",
    start=0.20,
    end=0.30,
    step=0.025,
    value=0.20,
    format='0.0%'
)


@pn.depends(rotor_diameter_dropdown, available_area_input, spacing_factor_slider, efficiency_slider)
def results_panel(rotor_diameter, available_area_km2, spacing_factor, efficiency):
    out = calculate_outputs(rotor_diameter, available_area_km2, spacing_factor, efficiency)

    # Original simple row: control + column of air density and wind speed
    widget_row = pn.Row(
        rotor_diameter_dropdown,
        pn.Column(
            pn.Row(
                pn.pane.Markdown("<b>Air Density (kg/m³):</b> {:.2f}".format(out['air_density'])),
                info_icon("Density of air at hub height")
            ),
            pn.Row(
                pn.pane.Markdown("<b>Wind Speed (m/s):</b> {:.2f}".format(out['wind_speed'])),
                info_icon("Average wind speed at hub height")
            )
        )
)

    # Section divider
    divider = pn.pane.HTML("<hr style='border:1px solid #bbb; margin:20px 0;'>")

    # Single Turbine Output row
    single_turbine_row = pn.Column(
        pn.pane.Markdown("## Single Turbine Output"),
        widget_row,
        pn.Row(efficiency_slider, info_icon("Overall efficiency/derating factor applied to annual energy")),
        pn.Row(
            pn.Column(
                pn.Row(
                    pn.pane.Markdown(f"**Swept Area:** {out['swept_area']:,.2f} m²"),
                    info_icon("Area swept by the turbine blades")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Annual Power Density:** {out['power_density']:,} W/m²"),
                    info_icon("Power per unit area from wind")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Power Output:** {out['power_kw']:,} kW"),
                    info_icon("Maximum power output of a single turbine")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Annual Energy Output (non-derated):** {out['energy_non_derated']:,} MWh/year"),
                    info_icon("Total annual energy output without losses")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Derated Annual Energy Output ({out['efficiency']*100:.1f}% efficiency):** {out['energy_derated']:,} MWh/year"),
                    info_icon("Annual energy output accounting for typical losses")
                )
            )
        )
)

    # Site Output row
    site_row = pn.Column(
        divider,
        pn.pane.Markdown("## Site Output"),
        available_area_input,
        pn.Row(spacing_factor_slider, info_icon("Spacing factor F (typical range: 3-10 for offshore wind farms)")),
        pn.Row(
            pn.Column(
                pn.Row(
                    pn.pane.Markdown(f"**Turbine Density Factor (F):** {spacing_factor:.2f}"),
                    info_icon("User-selected turbine density/spacing factor")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Turbine Spacing:** {out['turbine_spacing_m']:,.0f} m"),
                    info_icon("Center-to-center spacing = rotor diameter × F")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Installed Turbines:** {out['turbines']:,}"),
                    info_icon("Number of turbines that can be installed on site")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Total Power Output:** {out['site_power']:,} kW"),
                    info_icon("Total power output for all turbines on site")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Total Annual Energy Output (non-derated):** {out['site_energy_non_derated']:,} MWh/year"),
                    info_icon("Total annual energy output for the site without losses")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Total Derated Annual Energy Output ({out['efficiency']*100:.1f}% efficiency):** {out['site_energy_derated']:,} MWh/year"),
                    info_icon("Total annual energy output for the site accounting for typical losses")
                )
            )
        )
)

    return pn.Column(
        single_turbine_row,
        site_row
)


# Add a text title to the app
app_panel = pn.Column(
    pn.pane.Markdown("# Offshore wind calculations"),
    results_panel
)

In [3]:
pn.serve(app_panel)

Launching server at http://localhost:65190
